# Paralelni algoritmi - Prvi projekat

- Projekat se radi individualno ili u paru

- Rok za predaju projekata je **sreda 16.11.2022, do kraja dana (23:59)**. 

- Odbrane će biti organizovane u toku kolokvijumske nedelje.

- Predaja projekata je putem e-maila, programski kod bi trebalo da
bude prikacen uz sam e-mail. Subject e-maila treba da bude u formatu
"[RN3RA Projekat 1] Prezime (Prezime ukoliko se radi u paru)". Tekst e-maila treba da bude u formi:
"Ime Prezime XX/GGRN" (gde su XX i GG broje indeksa i godina upisa).
Link ka Collab-u je opcion. Ukoliko dva
studenta rade u paru, dovoljno je da jedan od njih preda, pri cemu
drugog stavi u cc.

### Ćelijski automati

Ćelijski automati su klasa modela koji se mogu koristiti za simulaciju različitih prirodnih fenomena. Korišćeni su za modeliranje razvoja tumora, kroz trodimenzionalnu mapu tkiva i predviđanje rasta ćelija kroz vreme. U neuronauci, ćelijski automati se mogu koristi za izučavanje populacije aktivacije neurona. Postoje mnogo druge primene, uključujući one u hemiji, biologiji, ekologiji i mnogim drugim granama nauke.

Ćelijski automati su u osnovi skup (prostorno) uređenih jedinica koje nazivamo ćelije. Svaka ćelija vrši neku od mogućih akcija, ili uzima neko od mogućih stanja, u zavisnosti od svoje neposredne okoline. Možda najpoznatiji primer ćelijskih automata je Konvejeva Igra života.

### Igra života

Konvejeva Igra života je ćelijski automat koji se sastoji iz pravougaone mreže ćelija. Svaka ćeilja može biti u jednom od sva stanja: *živa* i *mrtva*. Igra se odvija kroz vremenske iteracije, gde ćelije u jednoj iteraciji računaju svoje stanje u sledećoj. Stanje ćelije se računa na osnovu svog trenutnog stanja, trenutnog stanja svih neposrednih 8 suseda ćelije i sledećih pravila:
- Ako ćelija ima manje od 2 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako ćelija ima više od 3 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako je ćelija živa i ima 2 ili 3 živa suseda, u sledećoj iteraciji će biti živa
- Ako je ćelija mrtva i ima 3 živa suseda, u sledećoj iteraciji će biti živa

Osnovna struktura podataka Igre života je matrica stanja, koja za svaku ćeliju a[i, j] sadrži podatak da li je ćelija živa ili mrtva. Pri implementaciji igre javlja se problem suseda ćelija koje se nalaze uz ivicu mreže (ćelije u prvoj koloni nemaju "leve" susede, itd.). Postoji više rešenja za ovaj problem kao što su:
- Pretpostaviti da su nepostojeći susedi uvek mrtvi
- Uvesti "cikluse", tako da je poslednja kolona "levi" sused prve kolone, a poslednji red "gornji" sused prvog reda (efektivno torus).

Pri izradi zadataka koristiti **prvi** pristup.

### Zadatak
Zadatak je implementirati paralelizovanu verziju Igre života u programskom jeziku Python, na nekoliko načina. Prilikom implementacije obezbediti da se posle izvršavanja zadatog broja iteracija sačuva niz matrica stanja kroz vreme (stanja sistema u svakoj iteraciji), koji je kompatibilan sa datom funkcijom za animaciju (sledeća ćelija u fajlu).

1. Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom [10 poena]:  
  (a) Ključevima (threading.Lock)  
  (b) Ključevima uz upotrebu jednog uslova (threading.Condition)  
  (c) Ključevima uz upotrebu jedne barijere (threading.Condition)  
  Svaka nit simulira rad jedne ćelije u sistemu i ima pristup stanjima svojih suseda. 
  - Ćelije nemaju pristup globalnom brojaču iteracija, već svaka ćelija interno vodi računa o broju trenutne iteracije. 
  - Za svaku ćeliju postoji brojač suseda koji su pročitali njenu trenutnu vrednost. Ćelija koja je spremna da izmeni svoju vrednost periodično proverava da li su svi postojeći susedi pročitali njenu vrednost, u kom slučaju može da vrati brojač na nulu, izvrši izmenu i sačeka početak sledeće iteracije.
  - Iteracija se završava (i sledeća započinje) tako što:  
    (a) [4 poena] Niti treba da prođu kroz "čekaonicu", koju je potrebno implementirati uz pomoć dva brojača (celobrojne promeniljive) i jedne logičke (boolean) promeljive. 
     - Jedan brojač broji niti koje su ušle u čekaonicu (od 0 do N, gde je ukupan broj niti), a drugi broji niti koje su iz nje izašle (ovaj brojač se inicijalno postavlja na vrednost N).  
     - Prva nit koja hoće da uđe u čekaonicu (ulazni brojač == 0) prvo čeka da sve niti koje su prethodno ušle u čekaonicu izađu (izlazni brojač == N, pošto u suprotnom postoje niti iz prethodne iteracije koje nisu napustile čekaonicu), kada postavalja logičku promeniljivu na *False*, i ulazi u čekaonicu (ulazni brojač += 1).   
     - Poslednja nit koja uđe u čekaonicu (ulazni brojač == N) resetuje ulazni brojač, postavlja logički uslov na *True*, i izlazi iz čekaonice (postavlja izlazni brojač na 1).  
     - Ostale niti kad uđu u čekaonicu povećavaju ulazni brojač, čekaju da logički uslov bude postavljen na *True* posle čega izlaze (i povećavaju izlazni brojač).  

    (b) [3] poena Niti treba da prođu kroz "čekaonicu", koju je potrebno implementirati uz pomoć uslova (*Condition*) i jednog celobrojnog brojača. 
    - Svaka nit povećava brojač kada uđe u čekaonicu, i čeka na uslovu
    - Poslednja nit (brojač == N) resetuje brojač i budu sve niti koje čekaju na uslovu.

    (c) [3 poena] Niti treba da prođu kroz "čekaonicu", koju je potrebno implementirati uz pomoć barijere (*Barier*). 
  
2. Upotrebom procesa koji simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke (5 poena)  
  Svaki proces je simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju kroz iteracije razmenjuju putem redova za poruke. **Celokupna sinhronizacija mora da bude organizovana kroz poruke. Rešenja koja koriste mehanizme kao što su Lock, Condition i Barier se neće priznavati.**  
  Za potrebe analize rada implementirati poseban servis (dodatni proces) kojem sve ćelije javljaju novo stanje prilikom promene (pri čemu poruke sadrže koordinate ćelije, broj iteracije i novo stanje). Servis treba da rekonstruiše i sačuva (ili vrati u glavni program) niz matrica stanja.

3. Upotrebom više procesa kroz *process pool*, generisanjem taskova na nivou skupa ćelija (5 poena)  
  Matricu stanja podeliti na N delova (gde je N konfigurabilni parametar) i za svaki deo generisati *task* (poziv funkcije čijim parametrima se definiše koji deo matrice treba obraditi). Funkcija treba da vrati niz koordinata ćelija i njihove nove vrednosti, a matrica za sledeću iteraciju se može kreirati u glavnom programu. Trenutne vrednosti ćelija i suseda se mogu čitati iz deljene matrice.

1. (a)

In [ ]:
from threading import Thread, Lock
import time
import random
import sys
import numpy
from typing import List

state: numpy.ndarray
state_locks: numpy.ndarray
states: List[numpy.ndarray]
threads: numpy.ndarray

flag = False
in_waiting_room = 0
out_of_waiting_room = 0

mutex_flag = Lock()
mutex_in_waiting_room = Lock()
mutex_out_of_waiting_room = Lock()
empty = Lock()
full = Lock()

cell_visitors: numpy.ndarray
mutex_cell_visitors: numpy.ndarray

def cell(x, y, width, height, num_of_iterations):
    global state, states, state_locks, cell_visitors, mutex_cell_visitors, mutex_in_waiting_room, mutex_out_of_waiting_room, out_of_waiting_room, in_waiting_room, flag, mutex_flag

    adjacent_cells = [[-1,-1],[-1,0],[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1]]

    if(x == 0 and y == 0) :
        adjacent_cells = [[0,1],[1,1],[1,0]]
    elif(x == height - 1 and y == width - 1) :
        adjacent_cells = [[0,-1],[-1,-1],[-1,0]]
    elif(x == 0 and y == width - 1) :
        adjacent_cells = [[0,-1],[1,-1],[1,0]]
    elif(x == height - 1 and y == 0) :
        adjacent_cells = [[-1,0],[-1,1],[0,1]]
    elif(x == height - 1) :
        adjacent_cells = [[0,-1],[-1,-1],[-1,0],[-1,1],[0,1]]
    elif(y == width - 1) :
        adjacent_cells = [[-1,0],[-1,-1],[0,-1],[1,-1],[1,0]]
    elif(x == 0) :
        adjacent_cells = [[0,-1],[1,-1],[1,0],[1,1],[0,1]]
    elif(y == 0) :
        adjacent_cells = [[-1,0],[-1,1],[0,1],[1,1],[1,0]]


    for iteration in range(1, iteration_number + 1) :
        
        alive_neighbours = 0;

        for pair in adjacent_cells :
            xx = pair[0]
            yy = pair[1]

            state_locks[x + xx][y + yy].acquire()
            if(state[x + xx][y + yy] > 0) :
                alive_neighbours += 1
            state_locks[x + xx][y + yy].release()

            mutex_cell_visitors[x + xx][y + yy].acquire()
            cell_visitors[x + xx][y + yy] += 1
            mutex_cell_visitors[x + xx][y + yy].release()

        while(True) :
            mutex_cell_visitors[x][y].acquire()

            if( (cell_visitors[x][y] == 3 and ((x==0 and y==0 ) or (x==height-1 and y==width-1) or (x==0 and y==width-1) or (x==height-1 and y==0))) or
                (cell_visitors[x][y] == 5 and (x==0 or y==0 or x==height-1 or y==width-1)) or 
                (cell_visitors[x][y] == 8) ) :

                cell_visitors[x][y] = 0
                mutex_cell_visitors[x][y].release()
                break

            mutex_cell_visitors[x][y].release()
            time.sleep(0.01)
        
        state_locks[x][y].acquire()
        if(alive_neighbours < 2 or alive_neighbours > 3) :
            state[x][y] = 0
            states[iteration][x][y] = 0
        elif( (state[x][y] == 0 and alive_neighbours == 3) or 
              (state[x][y] == 1 and (alive_neighbours == 2 or alive_neighbours == 3)) ) :
            state[x][y] = 1
            states[iteration][x][y] = 1    
        state_locks[x][y].release()

        mutex_in_waiting_room.acquire()
        
        if(in_waiting_room == 0) : 

            while(True) :
                mutex_out_of_waiting_room.acquire()
                if(out_of_waiting_room >= width * height) :
                    mutex_out_of_waiting_room.release()
                    break
                mutex_out_of_waiting_room.release()
                time.sleep(0.01)

            mutex_flag.acquire()
            flag = False
            mutex_flag.release()

            in_waiting_room += 1
            mutex_in_waiting_room.release()
        
        elif(in_waiting_room >= width * height - 1) : 

            in_waiting_room = 0
            mutex_in_waiting_room.release()

            mutex_flag.acquire()
            flag = True
            mutex_flag.release()
                        
            mutex_out_of_waiting_room.acquire()
            out_of_waiting_room = 1
            mutex_out_of_waiting_room.release()
            continue            

        else :
            in_waiting_room += 1           
            mutex_in_waiting_room.release()
        
        while(True) :            
            mutex_flag.acquire()
            if(flag == True):
                mutex_flag.release()                
                break
            mutex_flag.release()
            time.sleep(0.01)

        mutex_out_of_waiting_room.acquire()
        out_of_waiting_room += 1
        mutex_out_of_waiting_room.release()
        

def generate_states(width=20, height=20, iteration_number=10):
    global state, state_locks, states, threads

    state = (numpy.random.rand(width * height).reshape(width, height) > 0.5).astype(numpy.int8)

height = 10
width = 10
iteration_number = 10

in_waiting_room = 0
out_of_waiting_room = height * width

state = (numpy.random.rand(width * height).reshape(width, height) > 0.5).astype(numpy.int8)
states = [numpy.zeros((width, height), dtype=int) for _ in range(iteration_number + 1)]
states[0] = state.copy()

cell_visitors = numpy.zeros((width, height), dtype=int)
mutex_cell_visitors = [[Lock() for _ in range(width)] for _ in range(height)]
state_locks = [[Lock() for _ in range(width)] for _ in range(height)]
threads = [[None for _ in range(width)] for _ in range(height)]

for i in range(height):
        for j in range(width):
            threads[i][j] = Thread(target=cell, args=(i, j, width, height, iteration_number))
            threads[i][j].start()

for i in range(height):
    for j in range(width):
        threads[i][j].join()

for i in range(iteration_number) :
    print(states[i], end="\n\n")


1. (b)

In [ ]:
from threading import Thread, Condition
import time
import random
import sys
import numpy
from typing import List

state: numpy.ndarray
states: List[numpy.ndarray]
threads: numpy.ndarray

counter = 0

condition = Condition()


def cell(x, y, width, height, num_of_iterations):
    global state, states, counter, condition

    adjacent_cells = [[-1,-1],[-1,0],[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1]]

    if(x == 0 and y == 0) :
        adjacent_cells = [[0,1],[1,1],[1,0]]
    elif(x == height - 1 and y == width - 1) :
        adjacent_cells = [[0,-1],[-1,-1],[-1,0]]
    elif(x == 0 and y == width - 1) :
        adjacent_cells = [[0,-1],[1,-1],[1,0]]
    elif(x == height - 1 and y == 0) :
        adjacent_cells = [[-1,0],[-1,1],[0,1]]
    elif(x == height - 1) :
        adjacent_cells = [[0,-1],[-1,-1],[-1,0],[-1,1],[0,1]]
    elif(y == width - 1) :
        adjacent_cells = [[-1,0],[-1,-1],[0,-1],[1,-1],[1,0]]
    elif(x == 0) :
        adjacent_cells = [[0,-1],[1,-1],[1,0],[1,1],[0,1]]
    elif(y == 0) :
        adjacent_cells = [[-1,0],[-1,1],[0,1],[1,1],[1,0]]


    for iteration in range(1, iteration_number + 1) :
        
        alive_neighbours = 0;
        for pair in adjacent_cells :
            xx = pair[0]
            yy = pair[1]

            if(state[x + xx][y + yy] > 0) :
                alive_neighbours += 1
        
        condition.acquire()

        counter += 1

        if(counter < height * width) :
            condition.wait()
        
        condition.notify()

        condition.release()

        if(alive_neighbours < 2 or alive_neighbours > 3) :
            state[x][y] = 0
            states[iteration][x][y] = 0
        elif( (state[x][y] == 0 and alive_neighbours == 3) or 
              (state[x][y] == 1 and (alive_neighbours == 2 or alive_neighbours == 3)) ) :
            state[x][y] = 1
            states[iteration][x][y] = 1

height = 10
width = 10
iteration_number = 10

state = (numpy.random.rand(width * height).reshape(width, height) > 0.5).astype(numpy.int8)
states = [numpy.zeros((width, height), dtype=int) for _ in range(iteration_number + 1)]
states[0] = state.copy()

cell_visitors = numpy.zeros((width, height), dtype=int)
threads = [[None for _ in range(width)] for _ in range(height)]

for i in range(height):
        for j in range(width):
            threads[i][j] = Thread(target=cell, args=(i, j, width, height, iteration_number))
            threads[i][j].start()

for i in range(height):
    for j in range(width):
        threads[i][j].join()

for i in range(iteration_number) :
    print(states[i], end="\n\n")

1. (c)

In [ ]:
from threading import Thread, Barrier
import time
import random
import sys
import numpy
from typing import List

state: numpy.ndarray
states: List[numpy.ndarray]
threads: numpy.ndarray

counter = 0


def cell(x, y, width, height, num_of_iterations):
    global state, states, counter, barrier

    adjacent_cells = [[-1,-1],[-1,0],[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1]]

    if(x == 0 and y == 0) :
        adjacent_cells = [[0,1],[1,1],[1,0]]
    elif(x == height - 1 and y == width - 1) :
        adjacent_cells = [[0,-1],[-1,-1],[-1,0]]
    elif(x == 0 and y == width - 1) :
        adjacent_cells = [[0,-1],[1,-1],[1,0]]
    elif(x == height - 1 and y == 0) :
        adjacent_cells = [[-1,0],[-1,1],[0,1]]
    elif(x == height - 1) :
        adjacent_cells = [[0,-1],[-1,-1],[-1,0],[-1,1],[0,1]]
    elif(y == width - 1) :
        adjacent_cells = [[-1,0],[-1,-1],[0,-1],[1,-1],[1,0]]
    elif(x == 0) :
        adjacent_cells = [[0,-1],[1,-1],[1,0],[1,1],[0,1]]
    elif(y == 0) :
        adjacent_cells = [[-1,0],[-1,1],[0,1],[1,1],[1,0]]


    for iteration in range(1, iteration_number + 1) :
        
        alive_neighbours = 0;
        for pair in adjacent_cells :
            xx = pair[0]
            yy = pair[1]

            if(state[x + xx][y + yy] > 0) :
                alive_neighbours += 1
        
        remaining = barrier.wait()

        if(remaining == 0) :
            barrier.reset()

        if(alive_neighbours < 2 or alive_neighbours > 3) :
            state[x][y] = 0
            states[iteration][x][y] = 0
        elif( (state[x][y] == 0 and alive_neighbours == 3) or 
              (state[x][y] == 1 and (alive_neighbours == 2 or alive_neighbours == 3)) ) :
            state[x][y] = 1
            states[iteration][x][y] = 1

height = 10
width = 10
iteration_number = 10

barrier = Barrier(height * width)

state = (numpy.random.rand(width * height).reshape(width, height) > 0.5).astype(numpy.int8)
states = [numpy.zeros((width, height), dtype=int) for _ in range(iteration_number + 1)]
states[0] = state.copy()

cell_visitors = numpy.zeros((width, height), dtype=int)
threads = [[None for _ in range(width)] for _ in range(height)]

for i in range(height):
        for j in range(width):
            threads[i][j] = Thread(target=cell, args=(i, j, width, height, iteration_number))
            threads[i][j].start()

for i in range(height):
    for j in range(width):
        threads[i][j].join()

for i in range(iteration_number) :
    print(states[i], end="\n\n")


U nastavku sledi kod koji možete koristi za vizuelizaciju rada automata.  
Upotreba ovog koda je opciona i ne nosi dodatne poene, ali je potrebno obezbediti nekakvu formu ispisa / vizuelizacije.

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np

def animate(steps):
  ''' Prima niz matrica (svaka matrica je stanje u jednom koraku simulacije) 
  prikazuje razvoj sistema'''
  
  def init():
    im.set_data(steps[0])
    return [im]
  
  
  def animate(i):
    im.set_data(steps[i])
    return [im]

  im = plt.matshow(steps[0], interpolation='None', animated=True);
  
  anim = FuncAnimation(im.get_figure(), animate, init_func=init,
                  frames=len(steps), interval=500, blit=True, repeat=False);
  return anim


n = 20
steps = [(np.random.rand(n**2).reshape(n, n) > 0.5).astype(np.int8)for i in range(50)]
anim = animate(steps);
HTML(anim.to_html5_video())

In [ ]:
a = np.zeros((20,5))